### crawler 만들기

자연어 수많은 언어 패턴 수집

데이터 마이닝 다양한 데이터 수집

*indexing -> 좋은 피처를 찾기위해 중요하다. 자연어 처리

focused crawling

starting url -> seed

url list -> que형식으로 메모리에 올려서 쓴다. - URL work pool handler/Work -thread manager

-> 나중에 DB로 바꾸는 작업까지

Scrapy -> 봇 vs 뷰솝 -> 파싱 목적

우리는 뷰숍으로 로우레벨로 접근할 수 있어야 한다.


---

http://example.webscraping.com/place/default/index

1. -> seed 주소지정

2. a태그의 href값 털기

In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup

In [285]:
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
def download(method, url, param=None, data=None):
    try:
        resp = requests.request(method, url,
                         params=None, data=None,
                         headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600\
            and maxretries > 0:
            print(maxretries)
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretries-1)
            print("재시도")
        else:
            print(e.response.status_code)
            print(e.response.reason)       
    return resp

In [7]:
url = "http://example.webscraping.com/place/default/index"

html = download("get",url)

In [9]:
dom = BeautifulSoup(html.text, "lxml")

In [10]:
[_["href"] for _ in dom.find_all("a") if len(_["href"])>3]

['/places/default/user/register?_next=/places/default/index',
 '/places/default/user/login?_next=/places/default/index',
 '/places/default/index',
 '/places/default/search',
 '/places/default/view/Afghanistan-1',
 '/places/default/view/Aland-Islands-2',
 '/places/default/view/Albania-3',
 '/places/default/view/Algeria-4',
 '/places/default/view/American-Samoa-5',
 '/places/default/view/Andorra-6',
 '/places/default/view/Angola-7',
 '/places/default/view/Anguilla-8',
 '/places/default/view/Antarctica-9',
 '/places/default/view/Antigua-and-Barbuda-10',
 '/places/default/index/1']

In [11]:
[requests.compat.urljoin(url,_["href"]) 
 for _ in dom.find_all("a") if len(_["href"])>3]

['http://example.webscraping.com/places/default/user/register?_next=/places/default/index',
 'http://example.webscraping.com/places/default/user/login?_next=/places/default/index',
 'http://example.webscraping.com/places/default/index',
 'http://example.webscraping.com/places/default/search',
 'http://example.webscraping.com/places/default/view/Afghanistan-1',
 'http://example.webscraping.com/places/default/view/Aland-Islands-2',
 'http://example.webscraping.com/places/default/view/Albania-3',
 'http://example.webscraping.com/places/default/view/Algeria-4',
 'http://example.webscraping.com/places/default/view/American-Samoa-5',
 'http://example.webscraping.com/places/default/view/Andorra-6',
 'http://example.webscraping.com/places/default/view/Angola-7',
 'http://example.webscraping.com/places/default/view/Anguilla-8',
 'http://example.webscraping.com/places/default/view/Antarctica-9',
 'http://example.webscraping.com/places/default/view/Antigua-and-Barbuda-10',
 'http://example.webscr

-- 1. row process(pdf)

In [129]:
def parseURL(seed):
    html = download("get",url)
    dom = BeautifulSoup(html.text,"lxml")
    
    return [requests.compat.urljoin(url,_["href"]) 
 for _ in dom.find_all("a") if _.has_attr("href") and len(_["href"])>3]

In [61]:
queue = list()

In [62]:
queue.append("http://example.webscraping.com/place/default/index")

In [63]:
import time

In [64]:
seen = list()
while queue:
    baseURL = queue.pop(0)
    print(baseURL)
    seen.append(baseURL)
    time.sleep(1) # 
    linkList = parseURL(baseURL)
    for link in linkList: #
        if link not in queue and link not in seen:
            queue.append(link)
            
    print("Queue : {0}, Seen : {1}".format(len(queue), len(seen)))
    # queue.extend(linkList)
    
    # -> time sleep , login/menu 메뉴 구성들은 사이트마다 중복될 것이다.
    # URL seen?

http://example.webscraping.com/place/default/index
Queue : 15, Seen : 1
http://example.webscraping.com/places/default/user/register?_next=/places/default/index
Queue : 14, Seen : 2
http://example.webscraping.com/places/default/user/login?_next=/places/default/index
Queue : 13, Seen : 3
http://example.webscraping.com/places/default/index
Queue : 12, Seen : 4
http://example.webscraping.com/places/default/search
Queue : 11, Seen : 5
http://example.webscraping.com/places/default/view/Afghanistan-1
Queue : 10, Seen : 6
http://example.webscraping.com/places/default/view/Aland-Islands-2
Queue : 9, Seen : 7
http://example.webscraping.com/places/default/view/Albania-3
Queue : 8, Seen : 8
http://example.webscraping.com/places/default/view/Algeria-4
Queue : 7, Seen : 9
http://example.webscraping.com/places/default/view/American-Samoa-5
Queue : 6, Seen : 10
http://example.webscraping.com/places/default/view/Andorra-6
Queue : 5, Seen : 11
http://example.webscraping.com/places/default/view/Angola-7


-- 2. 크롤러 만둘기

In [65]:
queue = list()
queue.append("https://www.google.com/search?ei=h98rXYOTLsWF8gWDgoa4BQ&q=%EB%B0%95%EB%B3%B4%EC%98%81&oq=%EB%B0%95%EB%B3%B4%EC%98%81&gs_l=psy-ab.3..35i39j0l9.151894.154727..154845...6.0..1.107.1339.1j12......0....1..gws-wiz.....10..0i71j35i39i19j0i10j0i131j0i10i42.-98QJTcPmIo")

In [66]:
seen = list()
baseURL = 0

In [67]:
len(queue)

1

In [69]:
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1) 
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)   
    print("Queue : {0}, Seen : {1}".format(len(queue), len(seen)))

In [59]:
a = ['http://example.webscraping.com/places/default/user/register?_next=/places/default/index', 'http://example.webscraping.com/places/default/user/login?_next=/places/default/index', 'http://example.webscraping.com/places/default/index', 'http://example.webscraping.com/places/default/search', 'http://example.webscraping.com/places/default/view/Afghanistan-1', 'http://example.webscraping.com/places/default/view/Aland-Islands-2', 'http://example.webscraping.com/places/default/view/Albania-3', 'http://example.webscraping.com/places/default/view/Algeria-4', 'http://example.webscraping.com/places/default/view/American-Samoa-5', 'http://example.webscraping.com/places/default/view/Andorra-6', 'http://example.webscraping.com/places/default/view/Angola-7', 'http://example.webscraping.com/places/default/view/Anguilla-8', 'http://example.webscraping.com/places/default/view/Antarctica-9', 'http://example.webscraping.com/places/default/view/Antigua-and-Barbuda-10', 'http://example.webscraping.com/places/default/index/1']

In [60]:
len(a)

15

In [76]:
html = download("get","https://search.naver.com/search.naver?where=image&sm=tab_jum&query=%EB%BD%80%EB%B8%94%EB%A6%AC")

In [77]:
dom= BeautifulSoup(html.text, "lxml")

In [78]:
[_["href"] for _ in dom.select(".photo_grid div a") if _["href"]!='#']

['http://www.derang.me/product/detail.html?product_no=13273',
 'https://tv.naver.com/v/8921526',
 'http://news.naver.com/main/read.nhn?oid=241&aid=0002811035',
 'http://sports.chosun.com/news/ntype.htm?id=201806190100152060011759&ServiceDate=20180618',
 'http://news.naver.com/main/read.nhn?oid=213&aid=0001103211',
 'http://news.naver.com/main/read.nhn?oid=076&aid=0003311356',
 'http://news.naver.com/main/read.nhn?oid=213&aid=0001052449',
 'http://www.seoul.co.kr/news/newsView.php?id=20180723801001',
 'https://cafe.naver.com/anycallusershow/2184427',
 'http://news.naver.com/main/read.nhn?oid=469&aid=0000382447',
 'http://news.naver.com/main/read.nhn?oid=109&aid=0003866322',
 'http://news.naver.com/main/read.nhn?oid=018&aid=0003375824',
 'http://news.naver.com/main/read.nhn?oid=213&aid=0001056904',
 'http://news.naver.com/main/read.nhn?oid=008&aid=0004081586',
 'https://blog.naver.com/yeji4772/220956554699',
 'http://news.naver.com/main/read.nhn?oid=382&aid=0000729106',
 'http://news.nav

In [79]:
queue = list()

In [82]:
queue.append(
    (_["href"] for _ in dom.select(".photo_grid div a") if _["href"]!='#')
)

In [83]:
queue

[['http://www.derang.me/product/detail.html?product_no=13273',
  'https://tv.naver.com/v/8921526',
  'http://news.naver.com/main/read.nhn?oid=241&aid=0002811035',
  'http://sports.chosun.com/news/ntype.htm?id=201806190100152060011759&ServiceDate=20180618',
  'http://news.naver.com/main/read.nhn?oid=213&aid=0001103211',
  'http://news.naver.com/main/read.nhn?oid=076&aid=0003311356',
  'http://news.naver.com/main/read.nhn?oid=213&aid=0001052449',
  'http://www.seoul.co.kr/news/newsView.php?id=20180723801001',
  'https://cafe.naver.com/anycallusershow/2184427',
  'http://news.naver.com/main/read.nhn?oid=469&aid=0000382447',
  'http://news.naver.com/main/read.nhn?oid=109&aid=0003866322',
  'http://news.naver.com/main/read.nhn?oid=018&aid=0003375824',
  'http://news.naver.com/main/read.nhn?oid=213&aid=0001056904',
  'http://news.naver.com/main/read.nhn?oid=008&aid=0004081586',
  'https://blog.naver.com/yeji4772/220956554699',
  'http://news.naver.com/main/read.nhn?oid=382&aid=0000729106',
 

In [139]:
url = "https://www.google.com/search"

In [140]:
param = {
    "q":"박보영"
}

In [141]:
html = download("get",url,param)

In [144]:
html.request.headers

{'User-Agent': 'python-requests/2.21.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Cookie': '1P_JAR=2019-07-15-02; NID=187=OSbjeH5k0DH4DHu0EJw109o8XIkrEruPPvirBWHCtEC3QDgYPdXl8WaAB4Y8HXMxGDynt_vcEjfqWx70scethIo52F3PmdfbJUMYYBU3PhthYkRVy909Dhyjg47Kfo-saJkSBFCkqOvJ-AWjf3e7sGCwDHl-kmGEsXXNmLBNSrY'}

In [145]:
dom = BeautifulSoup(html.text, "lxml")

In [146]:
print(dom.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"/><title>Google</title><script nonce="YvB6qCwIZW5TjiYVFNsNeA==">(function(){window.google={kEI:'WukrXdOrI8uA8QXV26OgDQ',kEXPI:'0,1353804,1957,2423,1225,730,224,510,1065,3151,57,322,206,397,620,175,1032,75,202,56,184,55,408,10,231,2331152,404,329140,1294,12383,4855,32692,15247,867,12163,5281,11240,363,3320,5505,2442,260,5107,575,835,284,2,579,727,2432,1361,4323,4967,774,2255,2816,1923,2650,250,223,2590,3601,669,1050,1808,1397,81,7,491,2044,7747,1162,5297,796,1222,36,920,746,7,120,1217,1211,68,85,346,1,1264,2736,3064,2,628,2403,837,44,1469,2,905,2406,2608,632,1139,1089,655,21,318,1117,475,427,104,1247,777,1,369,1317,79,625,756,98,392,30,399,992,509,598,10,170,6,109,1018,1495,174,967,48,553,11,14,12,411,856,1436,77

In [124]:
[_["href"] for _ in dom.select("div.r a")]

[]

In [130]:
queue = list()
queue.extend([_["href"] for _ in dom.select("div.r a")])
seen = list()
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1) 
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)   
    print("Queue : {0}, Seen : {1}".format(len(queue), len(seen)))
    

 --> 너비 우선 탐색 -> queue 구조

In [ ]:


while queue:
    baseURL = queue.pop(-1)#
    seen.append(baseURL)
    time.sleep(1) 
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)   
    print("Queue : {0}, Seen : {1}".format(len(queue), len(seen)))
    

--> 깊이 우선 탐색 -> stack 구조

In [153]:
url = "https://search.naver.com/search.naver"

In [154]:
param = {
    "query":"박보영"
}

In [157]:
html = download("get",url)

In [161]:
html.text

'<!doctype html> <html lang="ko"> <head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=2.0"> <meta property="og:title" content=" : 네이버 통합검색"/> <meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png"> <meta property="og:description" content="\'\'의 네이버 통합검색 결과입니다."> <meta name="description" lang="ko" content="\'\'의 네이버 통합검색 결과입니다."> <title> : 네이버 통합검색</title> <link rel="shortcut icon" href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_140327.ico">  <link rel="search" type="application/opensearchdescription+xml" href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" title="Naver" /><link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_190711.css"> <link rel="stylesheet" type="text/css" href="https://ssl.p

In [127]:
dom = BeautifulSoup(html.text, "lxml")

In [128]:
print(dom.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"/><title>Google</title><script nonce="Fky5hzBfxXtgxLu9/zWyUg==">(function(){window.google={kEI:'5-grXdinFcij8AXpwaCQDA',kEXPI:'0,1353804,1957,2423,1225,590,140,224,510,19,1046,3152,378,206,1017,1203,79,202,32,21,187,55,317,91,10,72,158,2331141,329556,1294,12383,4855,32692,15247,867,12163,16521,364,3319,5505,2442,260,5107,575,835,284,2,578,728,2432,1361,4323,4968,773,2251,2819,1924,3122,2591,3601,669,1050,1808,1397,81,7,491,2044,8909,5297,796,101,1119,38,920,746,7,120,1217,1364,346,1,1264,2736,3061,2,631,3248,36,2374,1774,635,1161,1446,632,2228,656,20,317,1119,474,430,101,1247,777,1,372,1315,703,756,98,392,29,400,992,98,1009,10,168,8,109,1018,1495,1141,48,553,11,14,10,1269,2212,25,177,25,298,7,1243,7,840,324,193,

In [148]:
queue = list()
queue.extend([_["href"] for _ in dom.select("._sp_each_url._sp_each_title")])
seen = list()

In [149]:
queue

[]

In [147]:
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1) 
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)   
    print("Queue : {0}, Seen : {1}".format(len(queue), len(seen)))

---

In [201]:
url = "https://blog.naver.com/shrnrgus80/221579714671"

In [193]:
html = download("get",url)

In [194]:
dom = BeautifulSoup(html.text, "lxml")

In [195]:
dom

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ko">
<head>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="noindex,follow" name="robots"/>
<meta content="always" name="referrer"/>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="/favicon.ico?2" rel="shortcut icon" type="image/x-icon"/>
<link href="https://rss.blog.naver.com/shrnrgus80.xml" rel="alternate" title="RSS feed for shrnrgus80 Blog" type="application/rss+xml"/>
<link href="https://blog.naver.com/NBlogWlwLayout.nhn?blogId=shrnrgus80" rel="wlwmanifest" type="application/wlwmanifest+xml"/>
<title>노로라의ForeverYoung : 네이버 블로그</title>
</head>
<script charset="UTF-8" src="https://ssl.pstatic.net/t.static.blog/mylog/versioning/Frameset-657805386_https.js" type="text/javascript"></script>
<script charse

In [170]:
[_ for _ in dom.select("div")]

[]

In [196]:
def checkBlog(url):
    return requests.compat.urlparse(url)[1] == "blog.naver.com"

In [207]:
def parseURL(seed):
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    if len(dom.select("#mainFrame")) < 1:
        return []
    
    seed = requests.compat.urljoin(seed,
                           dom.select("#mainFrame")[0]["src"])
    
    return [requests.compat.urljoin(seed,_["href"])
           for _ in dom.select("a")
           if _.has_attr("href") and len(_["href"])>3 and 
           checkBlog(requests.compat.urljoin(seed, _["href"]))]

In [227]:
dom.select("#mainFrame")

[<iframe allowfullscreen="true" id="mainFrame" name="mainFrame" onload="oFramesetTitleController.start(self.frames['mainFrame'], self, sTitle);oFramesetTitleController.onLoadFrame();oFramesetUrlController.start(self.frames['mainFrame']);oFramesetUrlController.onLoadFrame()" scrolling="auto" src="/PostView.nhn?blogId=shrnrgus80&amp;logNo=221579714671&amp;redirect=Dlog&amp;widgetTypeCall=true&amp;directAccess=false"></iframe>]

In [204]:
parseURL(url)

['https://blog.naver.com',
 'https://blog.naver.com/prologue/PrologueList.nhn?blogId=shrnrgus80',
 'https://blog.naver.com/PostView.nhn?blogId=shrnrgus80&logNo=221579714671&redirect=Dlog&widgetTypeCall=true&directAccess=false#blog-menu',
 'https://blog.naver.com/PostView.nhn?blogId=shrnrgus80&logNo=221579714671&redirect=Dlog&widgetTypeCall=true&directAccess=false#post_1',
 'https://blog.naver.com/MyBlog.nhn',
 'https://blog.naver.com/prologue/PrologueList.nhn?blogId=shrnrgus80',
 'https://blog.naver.com/prologue/PrologueList.nhn?blogId=shrnrgus80&skinType=&skinId=&from=menu',
 'https://blog.naver.com/PostList.nhn?blogId=shrnrgus80&skinType=&skinId=&from=menu',
 'https://blog.naver.com/PostList.nhn?blogId=shrnrgus80&categoryNo=12&skinType=&skinId=&from=menu',
 'https://blog.naver.com/PostList.nhn?blogId=shrnrgus80&categoryNo=11&skinType=&skinId=&from=menu',
 'https://blog.naver.com/PostList.nhn?blogId=shrnrgus80&categoryNo=1&skinType=&skinId=&from=menu',
 'https://blog.naver.com/PostLis

In [208]:
print(dom.prettify)

<bound method Tag.prettify of <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ko">
<head>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="noindex,follow" name="robots"/>
<meta content="always" name="referrer"/>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="/favicon.ico?2" rel="shortcut icon" type="image/x-icon"/>
<link href="https://rss.blog.naver.com/shrnrgus80.xml" rel="alternate" title="RSS feed for shrnrgus80 Blog" type="application/rss+xml"/>
<link href="https://blog.naver.com/NBlogWlwLayout.nhn?blogId=shrnrgus80" rel="wlwmanifest" type="application/wlwmanifest+xml"/>
<title>노로라의ForeverYoung : 네이버 블로그</title>
</head>
<script charset="UTF-8" src="https://ssl.pstatic.net/t.static.blog/mylog/versioning/Frameset-657805386_https.js" type="text/javasc

In [223]:
queue = list()

In [224]:
queue.extend(parseURL(url))

In [225]:
seen = list()

In [226]:
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1) 
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            print(link)
            queue.append(link)   
    print("Queue : {0}, Seen : {1}".format(len(queue), len(seen)))
    

Queue : 23, Seen : 1
Queue : 22, Seen : 2
Queue : 21, Seen : 3
Queue : 20, Seen : 4
Queue : 19, Seen : 5
Queue : 18, Seen : 6
Queue : 17, Seen : 7
Queue : 16, Seen : 8
Queue : 15, Seen : 9
Queue : 14, Seen : 10
Queue : 13, Seen : 11
Queue : 12, Seen : 12
Queue : 11, Seen : 13
Queue : 10, Seen : 14
Queue : 9, Seen : 15
Queue : 8, Seen : 16
https://blog.naver.com/prologue/PrologueList.nhn?blogId=shrnrgus80&&directAccess=true#blog-menu
https://blog.naver.com/prologue/PrologueList.nhn?blogId=shrnrgus80&&directAccess=true#post_1
https://blog.naver.com/PostView.nhn?blogId=shrnrgus80&logNo=221571576703&redirect=Dlog
https://blog.naver.com/PostView.nhn?blogId=shrnrgus80&logNo=221553305550&redirect=Dlog
https://blog.naver.com/PostView.nhn?blogId=shrnrgus80&logNo=221491592898&redirect=Dlog
https://blog.naver.com/PostView.nhn?blogId=shrnrgus80&logNo=221488532493&redirect=Dlog
https://blog.naver.com/PostView.nhn?blogId=shrnrgus80&logNo=221486827809&redirect=Dlog
https://blog.naver.com/PostView.nhn

 -> Focused crawling
 
 내부구조를 어느 정도 알아야 한다. iframe 이 들어있는 구조.

In [286]:
url = "https://adam24eve.tistory.com/858"
param = {
    "q":"박보영"
}
html = download("get", url, param)
dom = BeautifulSoup(html.text, "lxml")

In [317]:
def parseURL(seed):
    html = download("get",seed[0])
    dom = BeautifulSoup(html.text, "lxml")
    
#    if len(dom.select("#mainFrame")) < 1:
#        return []
    
#    seed = requests.compat.urljoin(seed,
#                           dom.select("#mainFrame")[0]["src"])
    
    return [(requests.compat.urljoin(seed,_["href"]),seed[1]+1)
           for _ in dom.select("a")
           if seed[1] < 2 and _.has_attr("href") and len(_["href"])
           and checkBlog(requests.compat.urljoin(seed[0], _["href"]))]
           #if _.has_attr("href") and len(_["href"])>3 and 
           #checkBlog(requests.compat.urljoin(seed, _["href"]))]

In [288]:
def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")

In [289]:
checkBlog("https://adam24eve.tistory.com")

True

In [295]:
print(dom.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<!--
  JB SKIN 109 Version 0.3.0
  https://www.jbfactory.net/
  # Open Source
    normalize.css v5.0.0    MIT License
    HTML5 Shiv v3.7.3       MIT/GPL2 License
    jQuery v1.12.3          MIT License
    modernizr v3.3.1        MIT License
    Placeholders.js v4.0.1  MIT License
		FontAwesome v4.7.0			http://fontawesome.io/license/
--><html lang="ko">
<head>
<link href="https://t1.daumcdn.net/tistory_admin/lib/lightbox/css/lightbox.min.css" rel="stylesheet" type="text/css"/><link href="https://t1.daumcdn.net/tistory_admin/assets/blog/cacc21797df8aa77ca75c9c748b07e9eb0ca9f80/blogs/style/content/font.css?_version_=cacc21797df8aa77ca75c9c748b07e9eb0ca9f80" rel="stylesheet" type="text/css"/><link href="https://t1.daumcdn.net/tistory_admin/assets/blog/cacc21797df8aa77ca75c9c748b07e9eb0ca9f80/blogs/style/content/content.css?_version_=cacc21797df8aa77ca75c9c748b07e9eb0ca9f80" rel="stylesheet" type="text/css"/><!--[if lt IE 9]><script src="https

In [318]:
[_["href"] for _ in dom.select('.link_item')]

['/category/Cosmos',
 '/category/Mystery%20world',
 '/category/Star',
 '/category/Political%20',
 '/category/It-game',
 '/category/Cosmos',
 '/category/Mystery%20world',
 '/category/Star',
 '/category/Political%20',
 '/category/It-game']

In [319]:
queue = list()

In [320]:
queue.extend([requests.compat.urljoin(url,_["href"]) for _ in dom.select('.link_item')])

In [321]:
queue

['https://adam24eve.tistory.com/category/Cosmos',
 'https://adam24eve.tistory.com/category/Mystery%20world',
 'https://adam24eve.tistory.com/category/Star',
 'https://adam24eve.tistory.com/category/Political%20',
 'https://adam24eve.tistory.com/category/It-game',
 'https://adam24eve.tistory.com/category/Cosmos',
 'https://adam24eve.tistory.com/category/Mystery%20world',
 'https://adam24eve.tistory.com/category/Star',
 'https://adam24eve.tistory.com/category/Political%20',
 'https://adam24eve.tistory.com/category/It-game']

In [322]:
seen = list()

In [323]:
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1) 
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            print(link)
            queue.append(link)   
    print("Queue : {0}, Seen : {1}".format(len(queue), len(seen)))

MissingSchema: Invalid URL 'h': No schema supplied. Perhaps you meant http://h?

-- link depth handling cf) SEO

In [270]:
import re

In [271]:
re.search(".+?\.tistory.com","http://adam24eve.tistory.com/")

<re.Match object; span=(0, 28), match='http://adam24eve.tistory.com'>